In [ ]:
import pandas as pd

# Load the CSV file into a pandas DataFrame
df_apps = pd.read_csv('data/apps_sample.csv')
df_transactions = pd.read_csv('data/txs_sample.csv')
# Specify the column name you want to analyze

# Get the number of unique entries for the specified column
unique_count_transactions = df_transactions['email'].nunique()
unique_count_apps = df_apps['email'].nunique()
print(f"The number of unique entries in the column '{'email'}' (txs_sample.csv): {unique_count_transactions}")
print(f"The number of unique entries in the column '{'email'}' (apps_sample.csv): {unique_count_apps}")


The number of unique entries in the column 'email' (txs_sample.csv): 2927
The number of unique entries in the column 'email' (apps_sample.csv): 3000


We can only use data that has 'Repaid' status, since it has the target that we need (balance after repayment).

In [25]:

# Get the count of each unique entry in the specified column
status_counts = df_apps['status'].value_counts()

print(f"Counts of each entry in the column '{'status'}':")
print(status_counts)

Counts of each entry in the column 'status':
status
Repaid          12322
AutoRejected     3021
Outstanding      1242
Expired           481
Cancelled         202
Open               14
Name: count, dtype: int64


In [32]:
# Extract all applications for a given customer
customer_email = '0521dc2fb6f29f79f126409112e21a975e6ed9a5bcc881fbd434045aa5d4a759'
customer_apps = df_apps[df_apps['email'] == customer_email]
print(f"Applications for the customer with email '{customer_email}':")
print(customer_apps[['applicationDate', 'amount', 'issuanceDate', 'status', 'paidAmount', 'repaidDate', 'nextPaycheck', 'paidByUser']])

Applications for the customer with email '0521dc2fb6f29f79f126409112e21a975e6ed9a5bcc881fbd434045aa5d4a759':
                applicationDate  amount              issuanceDate  \
13700  2024-11-11T04:28:04.411Z    35.0  2024-11-11T04:28:12.481Z   
14965  2024-11-29T12:31:53.133Z    15.0  2024-11-29T12:31:59.816Z   
17256  2024-12-31T14:51:02.540Z    20.0                       NaN   

             status  paidAmount                repaidDate nextPaycheck  \
13700        Repaid       45.24  2024-11-29T12:12:28.604Z   2024-11-20   
14965        Repaid       18.99  2024-12-13T17:47:30.594Z   2024-12-13   
17256  AutoRejected         NaN                       NaN   2025-01-10   

      paidByUser  
13700      False  
14965      False  
17256        NaN  


In [113]:
# Extract all transactions for a given customer
customer_email = '61be9c6909d91d74fe46a2c9f6d830d69f7015e8fa2ac4a4129cbb3edd18ae21'
df_transactions['date'] = pd.to_datetime(df_transactions['date'], format='%Y/%m/%d')
df_transactions['amount'] = pd.to_numeric(df_transactions['amount'], errors='coerce')

# Filter based on email and subcategory, then sort by date
filtered_by_email = df_transactions[df_transactions['email'] == customer_email]

# Then filter the resulting DataFrame based on the amount column
customer_txs = filtered_by_email[filtered_by_email['subcategory'].apply(lambda x: 'Micro Loan' in x)].sort_values(by='date')
print(f"All transactions for the customer with email '{customer_email}':")
# Only print amount, balance, category, subcategory, and date columns
print(customer_txs[['account_id', 'amount', 'balance', 'category', 'subcategory', 'tags', 'date']])


All transactions for the customer with email '61be9c6909d91d74fe46a2c9f6d830d69f7015e8fa2ac4a4129cbb3edd18ae21':
                                   account_id  amount  balance  \
1879953  6198e7ec-0e58-4e68-aa73-faa23a9fe30b  105.64 -1178.41   
1879954  6198e7ec-0e58-4e68-aa73-faa23a9fe30b  750.00  -712.77   
1879955  6198e7ec-0e58-4e68-aa73-faa23a9fe30b   20.00 -1072.77   
1879956  6198e7ec-0e58-4e68-aa73-faa23a9fe30b    7.77    37.23   
1879957  6198e7ec-0e58-4e68-aa73-faa23a9fe30b  -45.00    45.00   
...                                       ...     ...      ...   
2218820  c9511aa5-fc16-407a-bb45-b949a792f2c6 -160.00   138.72   
2218821  c9511aa5-fc16-407a-bb45-b949a792f2c6   21.27   -21.28   
2218822  c9511aa5-fc16-407a-bb45-b949a792f2c6   -0.13    -0.01   
2218823  c9511aa5-fc16-407a-bb45-b949a792f2c6   -0.16    -0.14   
2218824  c9511aa5-fc16-407a-bb45-b949a792f2c6    0.30    -0.30   

                              category                          subcategory  \
1879953        

In [154]:
import ast
unique_emails = df_transactions['email'].unique()
unique_emails_list = unique_emails.tolist()
df_transactions['date'] = pd.to_datetime(df_transactions['date'], errors='coerce').dt.date
df_apps['issuanceDate'] = pd.to_datetime(df_apps['issuanceDate'], errors='coerce').dt.date
df_apps['repaidDate'] = pd.to_datetime(df_apps['repaidDate'], errors='coerce').dt.date
def parse_tags(x):
    if pd.isna(x):  # Handle NaN
        return []
    try:
        return ast.literal_eval(x)  # Convert string to list
    except (ValueError, SyntaxError):  # Handle invalid formats
        return []


count = 0
for email, idx in zip(unique_emails_list, range(len(unique_emails_list))):
    # Filter based on email and subcategory, then sort by date
    # print(f"Email: {email}")
    if idx % 1000 == 0:
        print(f"Index: {idx}")
        print(f"Count: {count}")
    filtered_by_email = df_transactions[df_transactions['email'] == email]
    filtered_by_email['tags'] = filtered_by_email['tags'].apply(parse_tags)
    filtered_by_email['tags'] = filtered_by_email['tags'].apply(lambda x: x if isinstance(x, list) else [])
    customer_txs = filtered_by_email[
        filtered_by_email['tags'].apply(lambda x: 'Loan' in x)
    ].sort_values(by='date')
    apps_rows = df_apps[df_apps['email'] == email]
    for apps_index, apps_row in apps_rows.iterrows():
        if apps_row['status'] == 'Repaid':
            has_one = False
            for txs_index, txs_row in customer_txs.iterrows():
                if txs_row['amount'] == apps_row['amount']*-1 and txs_row['date'] == apps_row['issuanceDate'] and not has_one:
                    has_one = True
                elif has_one and txs_row['amount'] == apps_row['paidAmount'] and txs_row['date'] == apps_row['repaidDate']:
                    count += 1
                    break

print(count)
    # Only print amount, balance, category, subcategory, and date columns

Index: 0
Count: 0


/var/folders/mx/d620gz8j2_x32mmsnvc4bzkh0000gn/T/ipykernel_3060/146833991.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_by_email['tags'] = filtered_by_email['tags'].apply(parse_tags)
/var/folders/mx/d620gz8j2_x32mmsnvc4bzkh0000gn/T/ipykernel_3060/146833991.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_by_email['tags'] = filtered_by_email['tags'].apply(lambda x: x if isinstance(x, list) else [])


Index: 1000
Count: 3949
Index: 2000
Count: 5861
6486


In [155]:

count = 0
for email, idx in zip(unique_emails_list, range(len(unique_emails_list))):
    if idx % 1000 == 0:
        print(f"Index: {idx}")
        print(f"Count: {count}")
    filtered_by_email = df_transactions[df_transactions['email'] == email]
    filtered_by_email['tags'] = filtered_by_email['tags'].apply(parse_tags)
    filtered_by_email['tags'] = filtered_by_email['tags'].apply(lambda x: x if isinstance(x, list) else [])
    customer_txs = filtered_by_email[
        filtered_by_email['tags'].apply(lambda x: 'Loan' in x)
    ].sort_values(by='date')
    apps_rows = df_apps[df_apps['email'] == email]
    for apps_index, apps_row in apps_rows.iterrows():
        if apps_row['status'] == 'Repaid':
            has_one = False
            for txs_index, txs_row in customer_txs.iterrows():
                # if txs_row['amount'] == apps_row['amount']*-1 and txs_row['date'] == apps_row['issuanceDate'] and not has_one:
                #     has_one = True
                if txs_row['amount'] == apps_row['paidAmount'] and txs_row['date'] == apps_row['repaidDate']:
                    count += 1
                    break

print(count)
    # Only print amount, balance, category, subcategory, and date columns

Index: 0
Count: 0


/var/folders/mx/d620gz8j2_x32mmsnvc4bzkh0000gn/T/ipykernel_3060/3117788754.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_by_email['tags'] = filtered_by_email['tags'].apply(parse_tags)
/var/folders/mx/d620gz8j2_x32mmsnvc4bzkh0000gn/T/ipykernel_3060/3117788754.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_by_email['tags'] = filtered_by_email['tags'].apply(lambda x: x if isinstance(x, list) else [])


Index: 1000
Count: 5752
Index: 2000
Count: 8585
9489


Based on the 2 previous counts, we have more much more data with the transaction of just the loan repayment + balance instead of both transactions of obtaining the loan and repaying it back. So, to use more of the data, I will have to find another way to obtain the initial balance of the account (before the loan is given).